# LangServe

- LangServe는 LangChain에서 제공하는 **서비스 배포 프레임워크**임.

- LangChain 애플리케이션을 **REST API로 변환**하여 서비스화함.

- **Runnable 객체**를 API로 배포하고 **스트리밍 응답**을 지원함.

- **웹 기반 Playgroud**(API 테스트용)와 **클라이언트 SDK**를 제공함.


---

## 1. LangServe 설치하기

- LangServe 설치(pip 사용):

    ```bash
    pip install "langserve[all]"
    ```

- LangServe 설치(uv 사용):

    ```bash
    uv add "langserve[all]"   
    ```

- langchain-cli 설치:

    ```bash
    uv add langchain-cli
    ```

`(1) LangServe 설치 확인`

In [ ]:
import langserve
langserve.__version__

`(2) 환경 변수 로드`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

---
## 2. 간단한 LangServe 서버 만들기  

`(1) 서버 파일 생성`
- app 폴더에 별도 파일을 생성하여 LangServe 서버를 구현함. 
- server.py라는 이름으로 생성함.

    ```python
    # app/server.py
    from fastapi import FastAPI
    from langchain_openai import ChatOpenAI
    from dotenv import load_dotenv

    # 환경변수 로드
    load_dotenv()

    from langserve import add_routes

    # FastAPI 서버를 설정
    app = FastAPI(
        title="LangChain Server",
        version="1.0",
        description="Spin up a simple api server using Langchain's Runnable interfaces",
    )

    # 라우팅 설정
    add_routes(
        app,
        ChatOpenAI(model="gpt-4.1-mini"),
        path="/openai",   # OpenAI 모델에 대한 경로
    )

    if __name__ == "__main__":
        import uvicorn

        uvicorn.run(app, host="localhost", port=8000)
    ```

`(2) 서버 실행하기`


```bash
uv run python app/server.py
```

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0001.png)

`(3) Playground에서 테스트`

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0002.png)

`(4) LangSmith에서 트레이싱하기`

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0003.png)

---
## 3. LangServe 클라이언트 사용  

LangServe는 다음과 같은 API 엔드포인트를 자동으로 생성합니다:

- `/openai/invoke`: 동기 호출
- `/openai/stream`: 스트리밍 호출
- `/openai/batch`: 배치 처리
- `/openai/playground`: 웹 기반 UI 플레이그라운드

`(1) API 호출하기`

In [ ]:
# - `/openai/invoke`: 동기 호출

import requests

url = "http://localhost:8000/openai/invoke"
headers = {
    "Content-Type": "application/json",
}

data = {
    "input": "파이썬이란 무엇인가요?"
}

response = requests.post(url, headers=headers, json=data)
print(response.status_code)
print(response.json())

`(2) Python에서 클라이언트 SDK 사용하기`

In [ ]:
from langserve.client import RemoteRunnable

# 원격 실행 가능 객체를 생성
chain = RemoteRunnable("http://localhost:8000/openai")

# API 호출
response = chain.invoke("파이썬이란 무엇인가요?")
print(response)